# Creating a TorchModel
Base models built in PyTorch are already supported by LUME-model. We demonstrate how to create and execute a `TorchModel` below.

In [1]:
import torch

from lume_model.models import TorchModel, TorchModule
from lume_model.variables import ScalarVariable

## Building a Model from Scratch
Instantiation of a `TorchModel` requires specification of the base model (`torch.nn.Module`) and in-/output variables.

In [2]:
# exemplary model definition
base_model = torch.nn.Sequential(
    torch.nn.Linear(2, 1),
)
# variable specification
input_variables = [
    ScalarVariable(name="input1", default_value=0.1, value_range=[0.0, 1.0]),
    ScalarVariable(name="input2", default_value=0.2, value_range=[0.0, 1.0]),
]
output_variables = [
    ScalarVariable(name="output"),
]

# creation of TorchModel
example_model = TorchModel(
    model=base_model,
    input_variables=input_variables,
    output_variables=output_variables,
    fixed_model=True,
)

## Loading a Model from File

An already created model can be saved to a YAML file by calling the `dump` method. The model can then be loaded by simply passing the file to the constructor.

In [3]:
torch_model = TorchModel("../../tests/test_files/california_regression/torch_model.yml")
print(torch_model.yaml())

Loaded PyTorch model from file: Sequential(
  (0): Linear(in_features=8, out_features=24, bias=True)
  (1): ReLU()
  (2): Linear(in_features=24, out_features=12, bias=True)
  (3): ReLU()
  (4): Linear(in_features=12, out_features=6, bias=True)
  (5): ReLU()
  (6): Linear(in_features=6, out_features=1, bias=True)
)
model_class: TorchModel
input_variables:
  MedInc:
    variable_class: ScalarVariable
    default_value: 3.7857346534729004
    is_constant: false
    value_range: [0.4999000132083893, 15.000100135803223]
    value_range_tolerance: 1.0e-08
  HouseAge:
    variable_class: ScalarVariable
    default_value: 29.282135009765625
    is_constant: false
    value_range: [1.0, 52.0]
    value_range_tolerance: 1.0e-08
  AveRooms:
    variable_class: ScalarVariable
    default_value: 5.4074907302856445
    is_constant: false
    value_range: [0.8461538553237915, 141.90908813476562]
    value_range_tolerance: 1.0e-08
  AveBedrms:
    variable_class: ScalarVariable
    default_value: 1.10

## Model Execution and TorchModule
Calling the `evaluate` method allows for model execution on dictionary input. Additionally, instances of `TorchModel` can also be wrapped in a `TorchModule` which is a subclass of `torch.nn.Module`. This allows for seamless integration with `PyTorch` based packages like [BoTorch](https://botorch.org/) and [Xopt](https://christophermayes.github.io/Xopt/).

In [4]:
# generate exemplary input
input_dict = torch_model.random_input(n_samples=1)
input_dict

{'MedInc': tensor([0.5456]),
 'HouseAge': tensor([48.1874]),
 'AveRooms': tensor([110.0199]),
 'AveBedrms': tensor([12.6120]),
 'Population': tensor([18524.9258]),
 'AveOccup': tensor([530.1697]),
 'Latitude': tensor([40.4408]),
 'Longitude': tensor([-120.3350])}

In [5]:
# execute TorchModel
torch_model.evaluate(input_dict)

{'MedHouseVal': tensor(1.6094, dtype=torch.float64)}

In [6]:
# wrap in TorchModule
torch_module = TorchModule(model=torch_model)

In [7]:
# execute TorchModule
input_tensor = torch.tensor(
    [input_dict[k] for k in torch_module.input_order]
).unsqueeze(0)
torch_module(input_tensor)

tensor(1.6094, dtype=torch.float64)